In [1]:
import torchaudio
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC, Wav2Vec2CTCTokenizer
from transformers import Trainer, TrainingArguments
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from datasets import load_metric

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import librosa
from tqdm.notebook import tqdm
import glob
import os
import re
import json

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

/home/superceed1/anaconda3/envs/py38torch17/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


# Preprocessing

In [2]:
all_folders = glob.glob('data/*')

In [3]:
data = []
for folder in all_folders:
    wav_files = glob.glob(folder + '/wavs/*.wav')
    label_files = glob.glob(folder + '/labels/*.txt')
    
    for wav, label in zip(wav_files, label_files):
        text = open(label, 'r').read()
        if text != '':
            data.append({
                'wav_file': wav,
                'sentence': text
            })

In [4]:
df = pd.DataFrame(data)
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'
df['sentence'] = df['sentence'].apply(lambda x: re.sub(chars_to_ignore_regex, '', x).lower() + " ")

In [5]:
# Split train and test 
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=2021)

In [6]:
train_text = ' '.join(train_df['sentence']) 
eval_text = ' '.join(eval_df['sentence'])

vocab_list = list(set(train_text) | set(eval_text))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [7]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

41


In [8]:
with open('vocab/vocab.json', 'w') as json_file:
    json.dump(vocab_dict, json_file)

In [9]:
# Create tokenizer
tokenizer = Wav2Vec2CTCTokenizer('vocab/vocab.json', unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

# Feature Extraction

In [10]:
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0,
    do_normalize=True, return_attention_mask=True
)

In [11]:
processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer,
)

In [12]:
processor.save_pretrained('weights/wave2vec2-malay-processor')

In [13]:
# Resample from 41kHz to 16khz
# For that we need to create a dataset class from torch
# As malay is not part of the datasets class
class AudioDataSet(Dataset):
    def __init__(self, df, processor):
        self.df = df
        self.processor = processor
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        wav_file = self.df['wav_file'].iloc[idx]
        sentence = self.df['sentence'].iloc[idx]
        
        # load audio
        speech_array, sampling_rate = torchaudio.load(wav_file)
        speech = speech_array[0].numpy()
        
        # resample audio
        speech = librosa.resample(np.asarray(speech), sampling_rate, 16000)
        
        # prepare data
        input_values = self.processor(speech, sampling_rate=16000).input_values
        with self.processor.as_target_processor():
            labels = self.processor(sentence).input_ids
            
        return {
            'input_values': input_values[0],
            'labels': labels
        }

In [14]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None
        
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length = self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt"
        )
        
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt"
            )
            
        labels = labels_batch['input_ids'].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch['labels'] = labels
        
        return batch

In [15]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    
    return {'wer': wer}

In [16]:
train_dataset = AudioDataSet(train_df, processor)
eval_dataset = AudioDataSet(eval_df, processor)
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
wer_metric = load_metric('wer')

# Model & Train

In [17]:
model = Wav2Vec2ForCTC.from_pretrained(
    'facebook/wav2vec2-large-xlsr-53',
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
model.freeze_feature_extractor()

In [19]:
training_args = TrainingArguments(
    output_dir='weights/wave2vec2_bm/',
    group_by_length=True,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=30,
    fp16=False, # use fp32 for acc
    save_steps=400,
    eval_steps=400,
    learning_rate=3e-4,
    warmup_steps=500,
    save_total_limit=2
)

In [20]:
# Train
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor
)

In [ ]:
trainer.train()

Step,Training Loss
